# Overview

The purpose of this notebook is to injest all data needed for the project and write it into csv files for future processing. Some of the data is read in from csv files downloaded manually, while some of it is read in from the US Census Bureau API using a custom function. The data sets and their medium are:

* **CDC Diabetes Data:** Yearly diabetes prevelance data for each county, with an upper and lower limit of the estimate, for adults 20+ years. Data downloaded from the CDC website as separate csv files for each year & compiled here into one. 


* **ACS Demographic Data:** Yearly demographic data for counties with a population >65,000 from the US Census Bureau's American Community Survey (ACS) - variables such as age, race, education & median income. Queried from the US census API for each variable and year and compiled here.


* **All County Population Data:** Yearly estimates of the population of each US county, regardless of population size, from the Population Estimates Program (PEP). Queried from the US census API and compiled here.


* **All County Age Estimate Data:** A csv file with estimates of the age of the population in each US county and year between 2000-2010 provided by the US Census Bureau. Here the data is used to deterime the fraction of the population in each county 20 years and older in the year 2010; this is later used to approximate the number of adults with diabetes in each county for all years between 2006 and 2017 in the exploratory data analysis notebook.

# Code Navigation 
* [1. Load Packages](#1.LoadPackages)
* [2. CDC Diabetes Data](#2.CDCDiabetesData)
* [3. ACS Demographic Data](#3.ACSDemographicData)
* [4. All County Population Data](#4.AllCountyPopulationData)
* [5. All County Age Estimate Data](#5.AllCountyAgeEstimateData)
* [6. Write Data](#6.WriteData)

## 1. Load Packages <a class="anchor" id="1.LoadPackages"></a>

In [1]:
import requests
import pandas as pd
import csv
import os
import sys
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from tqdm import tqdm

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

#Change directory to project root directory
os.chdir("..")

from src.data.ACS_API import acs_api
from api_credentials import API_KEY
from src.data.PEP_API import pep_api
from src.data.ACS_variable_list import variable_list_2019, variable_list_2018, variable_list_2017, variable_list_2016, \
variable_list_2008, variable_list_2007, variable_list_2006

## 2. CDC Diabetes Data <a class="anchor" id="2.CDCDiabetesData"></a>

In [2]:
df_diabetes=pd.read_csv('data/raw/CDC_Data_yearly/First_csv/DiabetesAtlasCountyData.csv',skiprows=2,skipfooter=1,engine='python')
df_diabetes['Year']=2004

filenames = glob('data/raw/CDC_Data_yearly/*.csv')
for file in filenames:
    with open(file, newline='') as f:
        reader = csv.reader(f)
        row1 = next(reader)  # gets the first line
    year=row1[0].split(';')[-1].replace(' ','')
    df_diabetes_=pd.read_csv(file,skiprows=2,skipfooter=1,engine='python')
    df_diabetes_['Year']=year
    df_diabetes=pd.concat([df_diabetes_,df_diabetes])
df_diabetes=df_diabetes.sort_values(['State','County','Year'])
df_diabetes.reset_index(drop=True,inplace=True)

In [3]:
df_diabetes.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45164 entries, 0 to 45163
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   County        45164 non-null  object
 1   State         45164 non-null  object
 2   CountyFIPS    45164 non-null  int64 
 3   Percentage    45164 non-null  object
 4   Lower Limit   45164 non-null  object
 5    Upper Limit  45164 non-null  object
 6   Year          45164 non-null  object
dtypes: int64(1), object(6)
memory usage: 2.4+ MB


## 3. ACS Demographic Data <a class="anchor" id="3.ACSDemographicData"></a>

In [5]:
#Do initial call to create dataframe 
df_census=acs_api(API_KEY,requests,pd)
#Create list of years to query
years=[str(i+2006) for i in range(14)]
#Get data for all years and variables in one dataframe 
for year in tqdm(years):
    if int(year)==2019:
        variable_list=variable_list_2019
    if int(year)==2018:
        variable_list=variable_list_2018
    if int(year)==2017:
        variable_list=variable_list_2017
    if (int(year)<=2016) and (int(year)>=2010):
        variable_list=variable_list_2016
    if (int(year)==2009) or (int(year)==2008):
        variable_list=variable_list_2008
    if int(year)==2007:
        variable_list=variable_list_2007
    if int(year)==2006:
        variable_list=variable_list_2006
    for var_label,var_name in variable_list.items():
        df_census_=acs_api(API_KEY,requests,pd,var_label,var_name,year)
        df_census=pd.concat([df_census,df_census_])

100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [06:07<00:00, 26.22s/it]


In [6]:
df_census.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253773 entries, 0 to 839
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   county       253773 non-null  object
 1   state        253773 non-null  object
 2   value        253773 non-null  object
 3   state_fips   253773 non-null  object
 4   county_fips  253773 non-null  object
 5   year         253773 non-null  object
 6   variable     253773 non-null  object
dtypes: object(7)
memory usage: 15.5+ MB


In [7]:
df_census.head()

,county,state,value,state_fips,county_fips,year,variable
0,Baldwin County,Alabama,169162,01,003,2006,total_pop
1,Calhoun County,Alabama,112903,01,015,2006,total_pop
2,Cullman County,Alabama,80187,01,043,2006,total_pop
3,DeKalb County,Alabama,68014,01,049,2006,total_pop
4,Elmore County,Alabama,75688,01,051,2006,total_pop


## 4. All County Population Data <a class="anchor" id="4.AllCountyPopulationData"></a>

In [9]:
df_pop=pep_api(API_KEY,requests,pd,year='2000')
df_pop=pd.concat([df_pop,pep_api(API_KEY,requests,pd,year='2019')])

## 5. All County Age Estimate Data <a class="anchor" id="5.AllCountyAgeEstimateData">

In [11]:
#From https://www.census.gov/data/datasets/time-series/demo/popest/intercensal-2000-2010-counties.html
#2000-2010 Population/Age Estimates
df=pd.read_csv('data/raw/co-est00int-agesex-5yr.csv')

In [12]:
#Sex 0 means both sexes, Age=0 Means total population
total_pop_2010=df[(df.AGEGRP==0) & (df.SEX==0)][['STATE','COUNTY','STNAME','CTYNAME','POPESTIMATE2010']]
total_pop_2010=total_pop_2010.rename(columns={'POPESTIMATE2010':'total_population'})

In [13]:
#Sex 0 means both sexes, Age>4 means everyone 20+ years old
adult_pop_2010=pd.DataFrame(df[(df.AGEGRP>4) & (df.SEX==0)]
                    .groupby(['STATE','COUNTY','STNAME','CTYNAME'])
                    ['POPESTIMATE2010'].sum())
adult_pop_2010=adult_pop_2010.rename(columns={'POPESTIMATE2010':'adult_population'})
adult_pop_2010.head()

adult_population
STATE COUNTY STNAME  CTYNAME                         
1     1      Alabama Autauga County             38514
      3      Alabama Baldwin County            136821
      5      Alabama Barbour County             20754
      7      Alabama Bibb County                17130
      9      Alabama Blount County              41776

In [14]:
id_=['STATE','COUNTY','STNAME','CTYNAME']
df_age_population=adult_pop_2010.merge(total_pop_2010,right_on=id_,left_on=id_)
df_age_population['adult_pop_ratio']=df_age_population.adult_population/df_age_population.total_population
df_age_population.drop(columns=['adult_population','total_population'],inplace=True)
df_age_population.head()

,STATE,COUNTY,STNAME,CTYNAME,adult_pop_ratio
0,1,1,Alabama,Autauga County,0.704971
1,1,3,Alabama,Baldwin County,0.746860
2,1,5,Alabama,Barbour County,0.757141
3,1,7,Alabama,Bibb County,0.749114
4,1,9,Alabama,Blount County,0.728592


## 6. Write Data <a class="anchor" id="6.WriteData"></a>

In [17]:
df_diabetes.to_csv('data/raw/diabetes_data_2004_2017.csv')
df_pop.to_csv('data/raw/population_est_2000_2019.csv')
df_age_population.to_csv('data/raw/population_age_ratio_2010.csv')
df_census.to_csv('data/raw/ACS_data_2006_2019.csv')